In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler, PowerTransformer
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from xgboost import XGBRegressor
import optuna
import re
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import RFE

In [ ]:
class ClinicalTrialPredictor:
    def __init__(self):
        self.model = None
        self.feature_names = None
        self.target_transformer = PowerTransformer(method='yeo-johnson')
        self.text_vectorizer = None
        self.numeric_scaler = None

    def clean_text(self, text):
        if pd.isna(text):
            return ''
        text = re.sub(r'[^\w\s]', ' ', str(text))
        text = re.sub(r'\d+', 'NUM', text)
        text = re.sub(r'\s+', ' ', text)
        return text.strip().lower()

    def calculate_duration(self, start_date, completion_date):
        try:
            start = pd.to_datetime(start_date)
            end = pd.to_datetime(completion_date)
            return (end - start).days
        except:
            return np.nan

    def preprocess_data(self, trials_path, eligibilities_path):
        trials = pd.read_csv(trials_path, low_memory=False)
        chunksize = 50000
        relevant_eligibilities = []
        trial_ncts = set(trials['NCT Number'].unique())

        for chunk in pd.read_csv(
            eligibilities_path, 
            delimiter='|', 
            chunksize=chunksize, 
            low_memory=False,
            quoting=3, 
            on_bad_lines='warn',
        ):
            relevant_chunk = chunk[chunk['nct_id'].isin(trial_ncts)]
            relevant_eligibilities.append(relevant_chunk)

        eligibilities = pd.concat(relevant_eligibilities)
        data = trials.merge(eligibilities, left_on='NCT Number', right_on='nct_id', how='inner')
        data = data[(data['Study Type'] == 'INTERVENTIONAL') & (data['Study Status'] == 'COMPLETED')]

        data['Enrollment'] = pd.to_numeric(data['Enrollment'], errors='coerce')
        data = data[data['Enrollment'] > 0]

        data['Start Date'] = pd.to_datetime(data['Start Date'], errors='coerce')
        data = data.dropna(subset=['Start Date'])

        if len(data) > 100000:
            data = data.sample(n=100000, random_state=42)

        # Text Data Cleaning and Feature Extraction
        text_columns = ['Brief Summary', 'criteria', 'Study Title']
        text_data = data[text_columns].fillna('').agg(' '.join, axis=1)
        text_data = text_data.apply(self.clean_text)

        self.text_vectorizer = TfidfVectorizer(
            max_features=500,  
            stop_words='english',
            ngram_range=(1, 2),
            min_df=5
        )
        tfidf_features = self.text_vectorizer.fit_transform(text_data)

        svd = TruncatedSVD(n_components=100, random_state=42)  
        text_features = svd.fit_transform(tfidf_features)

        # Temporal and Numeric Features
        temporal_features = pd.DataFrame({
            'year': data['Start Date'].dt.year,
            'month': data['Start Date'].dt.month,
            'quarter': data['Start Date'].dt.quarter,
            'day_of_week': data['Start Date'].dt.dayofweek
        })

        data['criteria_length'] = data['criteria'].str.len()
        data['study_duration'] = data.apply(
            lambda x: self.calculate_duration(x['Start Date'], x['Completion Date']), 
            axis=1
        )

        for age_col in ['minimum_age', 'maximum_age']:
            data[f'{age_col}_years'] = data[age_col].apply(
                lambda x: float(str(x).replace('Years', '').strip()) if pd.notna(x) and 'Years' in str(x)
                else float(str(x).replace('Months', '').strip())/12 if pd.notna(x) and 'Months' in str(x)
                else np.nan
            )

        boolean_columns = ['adult', 'child', 'older_adult']
        for col in boolean_columns:
            data[col] = data[col].map({'t': 1, 'f': 0}).fillna(0)

        numeric_features = [
            'criteria_length', 'study_duration',
            'minimum_age_years', 'maximum_age_years',
            'adult', 'child', 'older_adult'
        ]

        numeric_data = data[numeric_features].fillna(data[numeric_features].mean())
        self.numeric_scaler = RobustScaler()
        numeric_scaled = self.numeric_scaler.fit_transform(numeric_data)

        # Combine Features
        X = np.hstack([
            text_features,
            numeric_scaled,
            temporal_features
        ])

        self.feature_names = (
            [f'text_{i}' for i in range(text_features.shape[1])] +
            numeric_features +
            temporal_features.columns.tolist()
        )

        y = data['Enrollment'].values
        return X, y

    def train_model(self, X_train, y_train):
        param_distributions = {
            'n_estimators': [100, 200, 300],
            'max_depth': [3, 5, 7, 10],
            'learning_rate': [0.01, 0.05, 0.1],
            'subsample': [0.8, 0.9, 1.0],
            'colsample_bytree': [0.8, 0.9, 1.0]
        }

        model = XGBRegressor(random_state=42, n_jobs=-1)
        random_search = RandomizedSearchCV(
            model,
            param_distributions,
            n_iter=50,  
            scoring='r2',
            cv=5,  
            verbose=1,
            n_jobs=-1,
            random_state=42
        )
        random_search.fit(X_train, y_train)

        self.model = random_search.best_estimator_
        return self.model

    def evaluate(self, X_test, y_test):
        y_pred = self.model.predict(X_test)
        metrics = {
            'rmse': mean_squared_error(y_test, y_pred, squared=False),
            'r2': r2_score(y_test, y_pred),
        }
        return metrics

    def run_pipeline(self, trials_path, eligibilities_path):
        X, y = self.preprocess_data(trials_path, eligibilities_path)
        y_transformed = np.log1p(y)  # Use log1p transformation

        X_train, X_test, y_train, y_test = train_test_split(
            X, y_transformed, test_size=0.2, random_state=42
        )
        self.train_model(X_train, y_train)
        metrics = self.evaluate(X_test, y_test)

        # Calculate MAPE
        y_pred = self.model.predict(X_test)
        mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
        metrics['mape'] = mape

        print("\nModel Evaluation:")
        print(f"RMSE: {metrics['rmse']:.2f}")
        print(f"R²: {metrics['r2']:.2f}")
        print(f"MAPE: {metrics['mape']:.2f}%")
        return metrics


In [ ]:
# Updated paths
TRIALS_PATH = "usecase_1_.csv"
ELIGIBILITIES_PATH = "eligibilities.txt"
clinical_predictor = ClinicalTrialPredictor()
metrics, importance = clinical_predictor.run_pipeline(TRIALS_PATH, ELIGIBILITIES_PATH)

In [ ]:
# in case of error plotting graphs please remove this function 
def visualize_results(self, metrics, feature_importance):
    # Metrics Visualization
    metrics_data = {
        'Metric': ['RMSE', 'R²', 'MAPE'],
        'Value': [metrics['rmse'], metrics['r2'], metrics['mape']]
    }
    metrics_df = pd.DataFrame(metrics_data)

    plt.figure(figsize=(8, 5))
    sns.barplot(data=metrics_df, x='Metric', y='Value', palette='coolwarm')
    plt.title('Model Performance Metrics', fontsize=14)
    plt.ylabel('Value', fontsize=12)
    plt.xlabel('Metric', fontsize=12)
    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)
    plt.tight_layout()
    plt.show()

    # Feature Importance Visualization
    top_features = feature_importance.head(10)

    plt.figure(figsize=(10, 6))
    sns.barplot(
        data=top_features, 
        y='feature', 
        x='importance', 
        palette='viridis'
    )
    plt.title('Top 10 Most Important Features', fontsize=14)
    plt.ylabel('Feature', fontsize=12)
    plt.xlabel('Importance', fontsize=12)
    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)
    plt.tight_layout()
    plt.show()

In [ ]:
Model Evaluation:
RMSE: 1.00
R²: 0.45
MAPE: 20.36%